In [4]:
import torch
import torch.nn as nn
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

#0 - prepare data

bc = datasets.load_breast_cancer()
X,y = bc.data, bc.target

n_samples, n_features = X.shape
#print(n_samples,n_features)

X_train, X_test,y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

#scale
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

X_train = torch.from_numpy(X_train.astype(np.float32))
X_test = torch.from_numpy(X_test.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))

#reshaping
y_train = y_train.view(y_train.shape[0], 1)
y_test = y_test.view(y_test.shape[0], 1)

#1 - model
#f = wx + b, sigmoid at the end
class LogisticRegression(nn.Module):
    
    def __init__(self,n_input_features):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(n_input_features, 1)   #1 is output
    
    def forward(self, x):
        y_predicted = torch.sigmoid(self.linear(x))
        return y_predicted

model = LogisticRegression(n_features)

#2 loss and opimizer
learning_rate = 0.01
criterion = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(),lr=learning_rate)

#3 training loop

num_epochs = 200
for epoch in range(num_epochs):
    # forwoard pass
    y_predicted = model(X_train)
    loss = criterion(y_predicted,y_train)
    
    #backward pass
    loss.backward()
    
    #update
    optimizer.step()
    
    #zero gradients
    optimizer.zero_grad()
    
    if (epoch+1) % 10 == 0:
        print(f'epoch: {epoch+1}, loss: {loss.item():.4f}')
        
with torch.no_grad():
    y_predicted = model(X_test)
    y_predicted_cls = y_predicted.round()
    acc = y_predicted_cls.eq(y_test).sum() / float(y_test.shape[0])
    print(f'accuracy = {acc:.4f}')
        

epoch: 10, loss: 0.6145
epoch: 20, loss: 0.4995
epoch: 30, loss: 0.4288
epoch: 40, loss: 0.3808
epoch: 50, loss: 0.3458
epoch: 60, loss: 0.3189
epoch: 70, loss: 0.2975
epoch: 80, loss: 0.2799
epoch: 90, loss: 0.2651
epoch: 100, loss: 0.2525
epoch: 110, loss: 0.2415
epoch: 120, loss: 0.2319
epoch: 130, loss: 0.2233
epoch: 140, loss: 0.2156
epoch: 150, loss: 0.2087
epoch: 160, loss: 0.2024
epoch: 170, loss: 0.1966
epoch: 180, loss: 0.1913
epoch: 190, loss: 0.1864
epoch: 200, loss: 0.1818
accuracy = 0.9298
